### Import Libraries

In [1]:
import arcpy
import pandas as pd
import numpy as np
import os
import censusdata as cs
from census import Census
pd.set_option('display.max_rows',None, 'display.max_columns',None,'display.max_colwidth', None)

## Set Directory

In [2]:
path = 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness'
os.chdir(path)

In [3]:
ProjectGDB =  r"C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\"
arcpy.env.overwriteOutput = True
arcpy.env.workspace = ProjectGDB

# Market Demand

- Business Analyst

In [ ]:
arcpy.ba.EnrichLayer(
    "ArlingtonMarketAssessmentAreas",
    "ArlingtonMarketAssessmentAreasEnriched",
    ############ Population Totals ################
    "populationtotals.TOTPOP_CY;populationtotals.POPDENS_CY;populationtotals.TOTPOP_FY;HistoricalHousing.TOTHU_CY; "\
    #######  Vacancy #####
    "vacant.VACANT_CY;vacant.VACANT_FY;housingunittotals.TOTHU_CY;homevalue.MEDVAL_CY;homevalue.ACSMEDVAL; "\
    ################# Retail ##################################
    "RetailDemandbyNAICS.INDRTFD_X;RetailDemandbyNAICS.INDRT_X;RetailDemandbyNAICS.IND722_X; "\
    ###################  Employment ###############################
    "EmploymentUnemployment.UNEMPRT_CY;KeyUSFacts.MEDVAL_CY;KeyUSFacts.MEDVAL_FY"\
    ################################## Crime#####################
    "crime.CRMCYTOTC"
    ,
    "",
    1,
    ""
)


- Residential Permits

In [4]:
ResidentialPermits = arcpy.management.SelectLayerByAttribute(
    "Issued_Permits",
    "NEW_SELECTION",
    """ SUBDESC IN ('Single-Family','Two-Family','Residential','Residential Pool Barrier','Residential Privacy',
        'Residential Spa','Residential Swimming Pool','Garage Conversion','Detached Garage',
        'Permanent Storage/Accy Building (over 320 sq ft)','Portable Storage/Accy Building (320 sq ft and under)',
        'Patio Cover/Gazebo','Carport','Retaining Wall' )
        
    AND WORKDESC IN ('Addition','Demolition', 'Foundation','Interior Finish','Move','New','New Construction',
    'New Construction - Shell','Remodel/Alterations/Repairs','Remove/Replace','Repair', 'Repair / Replacement','Reroof',
    'Solar Panel', 'Window/Door Replacement')
    """)

ResidentialPermcount = int(arcpy.management.GetCount(ResidentialPermits)[0])
print(ResidentialPermcount)

10463


In [5]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(ResidentialPermits)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('ResidentialPermits'):
    arcpy.Delete_management('ResidentialPermits')

arcpy.conversion.ExportFeatures(ResidentialPermits,"ResidentialPermits","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ResidentialPermits'>

- Near Analysis Residential Development

In [6]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketAssessmentAreasEnriched")
fieldmapping.addTable("ResidentialPermits")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketAssessmentAreasEnriched",
                           "ResidentialPermits","ArlingtonResNearby",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonResNearby","Join_Count","NearbyResidentialDevelopment","NearbyResidentialDevelopment")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonResNearby'>

- Commercial Permits

In [7]:
CommercialPermits = arcpy.management.SelectLayerByAttribute(
 "Issued_Permits",
    "NEW_SELECTION",
    """
    SUBDESC IN ('Assembly','Awning', 'Billboard', 'Boat Dock', 'Building Identification Sign', 'Business',
    'Canopy / Fascia Sign', 'Clean and Show', 'Commercial Privacy', 'Commercial Screening', 'Decks',
    'Directional',  'Directory',  'Educational',  'Entryway Sign',  'Existing Business/New Owner',
    'Expanding Lease Space',  'Factory',  'Ground',  'Institutional', 'Manufactured Housing',
    'Marquee Sign','Menu Board', 'Mercantile',  'Multi Tenant Ground Sign', 'New Tenant',
    'Onsite Development', 'Other Undefined Sign', 'Pole Sign','Projecting (Blade) Sign',
    'Same Business Owner/New Business Name','Shell Building/House Lights', 'Storage',
    'Subdivision Screening Wall', 'Temporary Construction Office', 'Temporary Sales Office',
    'Utility & Miscellaneous', 'Wall Sign'
    )
    AND WORKDESC IN ( 'New','New Construction','New Construction - Shell','Interior Finish',
    'Remodel/Alterations/Repairs', 'Repair', 'Repair / Replacement','Remove/Replace', 'Reroof',
    'Face Change', 'Foundation', 'Solar Panel', 'Window/Door Replacement','Demolition' )
    """
)

CommercialPermcount = int(arcpy.management.GetCount(CommercialPermits)[0])
print(CommercialPermcount)

4595


In [8]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(CommercialPermits)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('CommercialPermits'):
    arcpy.Delete_management('CommercialPermits')

arcpy.conversion.ExportFeatures(CommercialPermits,"CommercialPermits","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\CommercialPermits'>

- Near Analysis Commercial Development

In [9]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonResNearby")
fieldmapping.addTable("CommercialPermits")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonResNearby",
                           "CommercialPermits","ArlingtonComNearby",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonComNearby","Join_Count","NearbyCommercialDevelopment","NearbyCommercialDevelopment")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonComNearby'>

In [10]:
NewConstruction = arcpy.management.SelectLayerByAttribute(
 "Issued_Permits",
    "NEW_SELECTION",
    """
    WORKDESC IN ('New Construction - Shell', 'New Construction')
    """
)

NewConstructionCount = int(arcpy.management.GetCount(NewConstruction)[0])
print(NewConstructionCount)

3243


In [11]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(NewConstruction)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('NewConstruction'):
    arcpy.Delete_management('NewConstruction')

arcpy.conversion.ExportFeatures(NewConstruction,"NewConstruction","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\NewConstruction'>

- Near Analysis construction Activity

In [12]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonComNearby")
fieldmapping.addTable("NewConstruction")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonComNearby",
                           "NewConstruction","ArlingtonMarketReadiness",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadiness","Join_Count","NewConstructionActivity","NewConstructionActivity")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadiness'>

# Proximity to Amenity/Access

### Proximity to Schools
- Spatial join schools within 1 mile buffer to selected areas and count the number of schools

In [25]:
cols = []
for row in arcpy.ListFields('SchoolsPointsArlington'):
    cols.append(row.name)
cols  

['OBJECTID',
 'Shape',
 'DISTNAME',
 'CAMPNAME',
 'STREET',
 'CITY',
 'ZIP',
 'INSTR_TYPE',
 'XCOORD',
 'YCOORD',
 'DOWNLOADDATE',
 'GRADEGROUP',
 'State']

In [8]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadiness")
fieldmapping.addTable("SchoolsPointsArlington")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadiness",
                           "SchoolsPointsArlington","ArlingtonMarketReadinessSchools",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessSchools","Join_Count","NearbySchools","NearbySchools")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessSchools'>

### Proximity to Hospitals
- Spatial join hospitals within 1 mile buffer to selected areas and count the number of hospitals

In [9]:
cols = []
for row in arcpy.ListFields('Healthcare_Facilities'):
    cols.append(row.name)
cols  

['OBJECTID_1',
 'Shape',
 'OBJECTID',
 'BUILDINGCO',
 'ADDRESS',
 'HOSPITALNA',
 'Type']

In [11]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessSchools")
fieldmapping.addTable("Healthcare_Facilities")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessSchools",
                           "Healthcare_Facilities","ArlingtonMarketReadinessHealthcare",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessHealthcare","Join_Count","Nearbyhealthcare","Nearbyhealthcare")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessHealthcare'>

### Proximity to Grocery

In [12]:
cols = []
for row in arcpy.ListFields('Grocery'):
    cols.append(row.name)
cols  

['OBJECTID_1',
 'Shape',
 'ObjectId',
 'Store_Name',
 'Address',
 'City',
 'State',
 'County',
 'Longitude',
 'Latitude',
 'StoreType',
 'Produce']

In [13]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessHealthcare")
fieldmapping.addTable("Grocery")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessHealthcare",
                           "Grocery","ArlingtonMarketReadinessGrocery",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessGrocery","Join_Count","NearbyGrocery","NearbyGrocery")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessGrocery'>

### Proximity to Shopping Centers, Restaurants and Entertainment 
- This indicator only have data for Food estabishments and Downtown restuarants

In [17]:
cols = []
for row in arcpy.ListFields('DowntownRestuarants'):
    cols.append(row.name)
cols  

['OBJECTID',
 'Shape',
 'Type',
 'Tags',
 'ServesAlcohol',
 'Name',
 'Address',
 'City',
 'State',
 'ZIP',
 'Phone',
 'OnlineReviewLink',
 'NavigateToLink',
 'CreationDate',
 'Creator',
 'EditDate',
 'Editor']

In [18]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessGrocery")
fieldmapping.addTable("FoodEstablishmentDowntownRestaurants")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessGrocery",
                           "FoodEstablishmentDowntownRestaurants","ArlingtonMarketReadinessFoodEstablishments",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessFoodEstablishments","Join_Count","NearbyRestuarants",
                            "NearbyRestuarants")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessFoodEstablishments'>

### Proximity to Parks

- Converted the Parks Polygon to point and counted the number of parks that are in proximity to the selected areas

In [19]:
arcpy.management.FeatureToPoint("Park2024","ArlingtonParks2024","CENTROID")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonParks2024'>

In [22]:
cols = []
for row in arcpy.ListFields('ArlingtonParks2024'):
    cols.append(row.name)
cols  

['OBJECTID_1',
 'Shape',
 'OBJECTID',
 'NAME',
 'OWNER',
 'NOTES',
 'CITY',
 'FEATTYPE',
 'STATUS',
 'SOURCE',
 'SOURCE_YEA',
 'ShapeSTLen',
 'ShapeSTAre',
 'ORIG_FID']

In [23]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessFoodEstablishments")
fieldmapping.addTable("ArlingtonParks2024")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor','OBJECTID_1','Shape','OBJECTID','NAME','OWNER','NOTES','CITY',
                  'FEATTYPE','STATUS','SOURCE', 'SOURCE_YEA', 'ShapeSTLen', 'ShapeSTAre', 'ORIG_FID']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessFoodEstablishments",
                           "ArlingtonParks2024","ArlingtonMarketReadinessParks",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessParks","Join_Count","NearbyParks","NearbyParks")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessParks'>

- Walkability

In [35]:
# Intersect Selected Areas and EPA NWI

arcpy.analysis.Intersect("ArlingtonMarketReadinessParks #;NationalWalkabilityIndex #",
                         "ArlingtonMarketReadiness_Intersect","ALL",None,"INPUT")


# Add Field to calculated weighted area

arcpy.management.AddField("ArlingtonMarketReadiness_Intersect","WeightedWalk","DOUBLE",None,None,None,"WeightedWalk",
                          "NULLABLE","NON_REQUIRED","")

# Compute the Weighted Walk

arcpy.management.CalculateField("ArlingtonMarketReadiness_Intersect","WeightedWalk","!Acres! * !NatWalkInd!",
                                "PYTHON3","","DOUBLE","NO_ENFORCE_DOMAINS")


# Create summary statistics, the Ranking is the Id

arcpy.analysis.Statistics("ArlingtonMarketReadiness_Intersect","WalkabilityStatistics",
                          "WeightedWalk SUM;Acres SUM","Ranking","")

# Join summary to Market Readiness data

walkability = arcpy.management.AddJoin("ArlingtonMarketReadinessParks","Ranking","WalkabilityStatistics",
                         "Ranking","KEEP_ALL","NO_INDEX_JOIN_FIELDS","NO_REBUILD_INDEX")

arcpy.conversion.ExportFeatures(walkability,"ArlingtonMarketReadinessEPAWI",
                                "","NOT_USE_ALIAS",'',None)

# Add Field to calculated weighted area

arcpy.management.AddField("ArlingtonMarketReadinessEPAWI","WalkabilityIndex","DOUBLE",None,None,None,"WalkabilityIndex",
                          "NULLABLE","NON_REQUIRED","")

# Compute the Weighted Walk

arcpy.management.CalculateField("ArlingtonMarketReadinessEPAWI","WalkabilityIndex","!SUM_WeightedWalk! / !SUM_Acres!",
                                "PYTHON3","","DOUBLE","NO_ENFORCE_DOMAINS")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadiness_Intersect'>

In [ ]:
cols = []
for row in arcpy.ListFields('ArlingtonMarketReadinessEPAWI'):
    cols.append(row.name)
cols  

- Distance to Interstate/Major Highways

In [87]:
arcpy.analysis.Near("ArlingtonMarketReadinessEPAWI","TxDOT_Roadway",None,"NO_LOCATION","NO_ANGLE","PLANAR",
                    "NEAR_DIST NEAR_DIST","Miles")

arcpy.management.AlterField("ArlingtonMarketReadinessEPAWI","NEAR_DIST","DistanceToHighway","DistanceToHighway")


<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessEPAWI'>

- Distance to Regional Rail

In [88]:
arcpy.analysis.Near("ArlingtonMarketReadinessEPAWI","Texas_Railroads",None,"NO_LOCATION","NO_ANGLE","PLANAR",
                    "NEAR_DIST NEAR_DIST","Miles")

arcpy.management.AlterField("ArlingtonMarketReadinessEPAWI","NEAR_DIST","DistanceToRegionalRail","DistanceToRegionalRail")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessEPAWI'>

- Distance to DFW/Love Field/Grand Praire

In [90]:
arcpy.analysis.Near("ArlingtonMarketReadinessEPAWI","Airport_Boundary",None,"NO_LOCATION","NO_ANGLE","PLANAR",
                    "NEAR_DIST NEAR_DIST","Miles")

arcpy.management.AlterField("ArlingtonMarketReadinessEPAWI","NEAR_DIST","DistanceToAirport","DistanceToAirport")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessEPAWI'>

# Infrastructure Capacity

#### Fire and police station
- merged the fire and police station points, then computed the count of each stations within the selected areas


In [24]:
cols = []
for row in arcpy.ListFields('FireandPoliceStations'):
    cols.append(row.name)
cols  

['OBJECTID_1',
 'Shape',
 'OBJECTID',
 'STATIONNAM',
 'PARCELADDR',
 'STRUCTUREA',
 'DISTRICTID',
 'STATION']

In [65]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessEPAWI")
fieldmapping.addTable("FireandPoliceStations")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor','OBJECTID_1','Shape','OBJECTID','NAME','OWNER','NOTES','CITY',
                  'FEATTYPE','STATUS','SOURCE', 'SOURCE_YEA', 'ShapeSTLen', 'ShapeSTAre', 'ORIG_FID',
                 'OBJECTID_1','Shape','OBJECTID','STATIONNAM','PARCELADDR','STRUCTUREA','DISTRICTID','STATION',
                 'Walkscore','OBJECTID','Ranking_1','FREQUENCY','SUM_WeightedWalk','SUM_Acres','Shape_Length','Shape_Area',
                 'NEAR_FID']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessEPAWI",
                           "FireandPoliceStations","ArlingtonMarketReadinessPoliceFireStations",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessPoliceFireStations","Join_Count","NearbyPolice","NearbyPolice")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessPoliceFireStations'>

In [67]:
cols = []
for row in arcpy.ListFields('ArlingtonMarketReadinessPoliceFireStations'):
    cols.append(row.name)
cols  

['OBJECTID',
 'Shape',
 'NearbyPolice',
 'TARGET_FID',
 'NearbyParks',
 'NearbyRestuarants',
 'NearbyGrocery',
 'Nearbyhealthcare',
 'NearbySchools',
 'NewConstructionActivity',
 'NearbyCommercialDevelopment',
 'NearbyResidentialDevelopment',
 'TARGET_FID_12',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Ranking',
 'Location',
 'Value',
 'Override',
 'aggregationMethod',
 'HasData',
 'ORIGINAL_OID',
 'sourceCountry',
 'apportionmentConfidence',
 'populationToPolygonSizeRating',
 'populationtotals_TOTPOP_CY',
 'populationtotals_POPDENS_CY',
 'populationtotals_TOTPOP_FY',
 'HistoricalHousing_TOTHU_CY',
 'vacant_VACANT_CY',
 'vacant_VACANT_FY',
 'housingunittotals_TOTHU_CY',
 'homevalue_MEDVAL_CY',
 'homevalue_ACSMEDVAL',
 'RetailDemandbyNAICS_INDRTFD_X',
 'RetailDemandbyNAICS_INDRT_X',
 'RetailDemandbyNAICS_IND722_X',
 'EmploymentUnemployment_UNEMPRT_CY',
 'KeyUSFacts_MEDVAL_CY',
 'KeyUSFacts_MEDVAL_FY',
 'WalkabilityIndex',
 'Shape_Length',
 'Shape_Area']

- Traffic Count

In [68]:
cols = []
for row in arcpy.ListFields('TrafficCountyHistoric5Year'):
    cols.append(row.name)
cols  

['OBJECTID',
 'Shape',
 'DIST_NM',
 'CNTY_NM',
 'TRFC_STATN',
 'LATEST_AAD',
 'AADT_RPT_Q',
 'AADT_RPT_H',
 'AADT_RPT_1',
 'AADT_RPT_2',
 'AADT_RPT_3']

In [83]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessPoliceFireStations")
fieldmapping.addTable("TrafficCountyHistoric5Year")

index = fieldmapping.findFieldMapIndex('AADT_RPT_Q')
fmap = fieldmapping.getFieldMap(index)
fmap.mergeRule = 'Mean'
fieldmapping.replaceFieldMap(index, fmap)


analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor','OBJECTID_1','Shape','OBJECTID','NAME','OWNER','NOTES','CITY',
                  'FEATTYPE','STATUS','SOURCE', 'SOURCE_YEA', 'ShapeSTLen', 'ShapeSTAre', 'ORIG_FID',
                 'OBJECTID_1','Shape','OBJECTID','STATIONNAM','PARCELADDR','STRUCTUREA','DISTRICTID','STATION',
                 'Walkscore','OBJECTID','Ranking_1','FREQUENCY','SUM_WeightedWalk','SUM_Acres','Shape_Length','Shape_Area',
                 'OBJECTID','Shape','DIST_NM','CNTY_NM','TRFC_STATN','LATEST_AAD', 'AADT_RPT_H','TARGET_FID_12',
                  'AADT_RPT_1', 'AADT_RPT_2','AADT_RPT_3','Join_Count','TARGET_FID']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessPoliceFireStations",
                           "TrafficCountyHistoric5Year","ArlingtonMarketReadinessTraffic",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessTraffic","AADT_RPT_Q","Avg5yrTraffic","Avg5yrTraffic")
arcpy.management.AlterField("ArlingtonMarketReadinessTraffic","Join_Count","NearbyTrafficSensors","NearbyTrafficSensors")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessTraffic'>

In [82]:
cols = []
for row in arcpy.ListFields('ArlingtonMarketReadinessTraffic'):
    cols.append(row.name)
cols 

['OBJECTID',
 'Shape',
 'NearbyTrafficCensors',
 'TARGET_FID',
 'NearbyPolice',
 'NearbyParks',
 'NearbyRestuarants',
 'NearbyGrocery',
 'Nearbyhealthcare',
 'NearbySchools',
 'NewConstructionActivity',
 'NearbyCommercialDevelopment',
 'NearbyResidentialDevelopment',
 'Shape_Leng',
 'Comments',
 'Acres',
 'Descriptio',
 'Ranking',
 'Location',
 'Value',
 'Override',
 'aggregationMethod',
 'HasData',
 'ORIGINAL_OID',
 'sourceCountry',
 'apportionmentConfidence',
 'populationToPolygonSizeRating',
 'populationtotals_TOTPOP_CY',
 'populationtotals_POPDENS_CY',
 'populationtotals_TOTPOP_FY',
 'HistoricalHousing_TOTHU_CY',
 'vacant_VACANT_CY',
 'vacant_VACANT_FY',
 'housingunittotals_TOTHU_CY',
 'homevalue_MEDVAL_CY',
 'homevalue_ACSMEDVAL',
 'RetailDemandbyNAICS_INDRTFD_X',
 'RetailDemandbyNAICS_INDRT_X',
 'RetailDemandbyNAICS_IND722_X',
 'EmploymentUnemployment_UNEMPRT_CY',
 'KeyUSFacts_MEDVAL_CY',
 'KeyUSFacts_MEDVAL_FY',
 'WalkabilityIndex',
 'Avg5yrTraffic',
 'Shape_Length',
 'Shape_Are

- **Environmental and Protected Areas**

In [85]:
cols = []
for row in arcpy.ListFields('Brownfields'):
    cols.append(row.name)
cols 

['OBJECTID_1',
 'Shape',
 'OBJECTID',
 'TaxParcelI',
 'TaxParcelA',
 'TaxLegalKe',
 'MoreInform',
 'ShapeSTAre',
 'ShapeSTLen',
 'Shape_Length',
 'Shape_Area']

In [84]:
arcpy.management.FeatureToPoint("Brownfields","Brownfields2024","CENTROID")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\Brownfields2024'>

In [86]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketReadinessTraffic")
fieldmapping.addTable("Brownfields")

analysisfields = ['OBJECTID','Shape','DISTNAME','CAMPNAME','STREET','CITY','ZIP','INSTR_TYPE','XCOORD','YCOORD',
                  'DOWNLOADDATE','GRADEGROUP','State','TARGET_FID','TARGET_FID_1','ObjectID_1','OBJECTID_1','Shape',
                  'OBJECTID','BUILDINGCO','ADDRESS','HOSPITALNA','Type','OBJECTID_1','Shape','ObjectId','Store_Name',
                  'Address','City','State','County','Longitude','Latitude','StoreType','Produce',
                 'OBJECTID_1','Shape','ImportDate','OBJECTID','FacilityNa','Establishm','PropertyAd',
                  'Inspection','Inspecti_1','Attempt','XCoord','YCoord','Inspecti_2', 'Type','Tags','ServesAlcohol',
                  'Name','Address','City','State','ZIP','Phone','OnlineReviewLink','NavigateToLink','CreationDate',
                  'Creator','EditDate','Editor','OBJECTID_1','Shape','OBJECTID','NAME','OWNER','NOTES','CITY',
                  'FEATTYPE','STATUS','SOURCE', 'SOURCE_YEA', 'ShapeSTLen', 'ShapeSTAre', 'ORIG_FID',
                 'OBJECTID_1','Shape','OBJECTID','STATIONNAM','PARCELADDR','STRUCTUREA','DISTRICTID','STATION',
                 'Walkscore','OBJECTID','Ranking_1','FREQUENCY','SUM_WeightedWalk','SUM_Acres','Shape_Length','Shape_Area',
                 'OBJECTID_1', 'Shape','OBJECTID', 'TaxParcelI', 'TaxParcelA', 'TaxLegalKe', 'MoreInform', 'ShapeSTAre',
                  'ShapeSTLen']


for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessTraffic",
                           "Brownfields","ArlingtonMarketReadinessBrownfields",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadinessBrownfields","Join_Count","NearbyBrownfields","NearbyBrownfields")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadinessBrownfields'>

- **Road Conditions**

In [105]:
## Add field to road segment 

arcpy.management.AddField("StreetCondition2024","segmentlength","DOUBLE",None,None,None,"segmentlength",
                          "NULLABLE","NON_REQUIRED","")


arcpy.management.CalculateGeometryAttributes(
    in_features="StreetCondition2024",
    geometry_property=[["segmentlength", "LENGTH_GEODESIC"]],
    length_unit="MILES_US",
    coordinate_system=arcpy.SpatialReference(32038)  # NAD 1983 StatePlane Texas North Central Feet
)

# Add fields for Weighted length

arcpy.management.AddField("StreetCondition2024", "OCIxLen", "DOUBLE")

# Calculate field 

arcpy.management.CalculateField("StreetCondition2024", "OCIxLen", "!EstimatedOCI! * !segmentlength!", "PYTHON3")

#  Spatially join street condtion layer to last brownfields layer
arcpy.analysis.SpatialJoin("ArlingtonMarketReadinessBrownfields","StreetCondition2024",
                           "ArlingtonMarketReadinessStreetcondt","JOIN_ONE_TO_MANY",
                           "KEEP_COMMON",'',"INTERSECT","1 Miles","",None)

# Grab summary statistics 

arcpy.analysis.Statistics("ArlingtonMarketReadinessStreetcondt","ArlingtonMarketReadinessStreetConditionSummary",
                          [["OCIxLen", "SUM"], ["segmentlength", "SUM"]],
                          "Ranking"  )

# Add field to for weighted OCI

arcpy.management.AddField("ArlingtonMarketReadinessStreetConditionSummary", "WeightedOCI", "DOUBLE")

# Calculate field

arcpy.management.CalculateField("ArlingtonMarketReadinessStreetConditionSummary","WeightedOCI",
                                "!SUM_OCIxLen! / !SUM_segmentlength!","PYTHON3")

# Add street condition 

streetcondition = arcpy.management.AddJoin("ArlingtonMarketReadinessBrownfields","Ranking",
                                           "ArlingtonMarketReadinessStreetConditionSummary","Ranking",
                                           "KEEP_ALL","NO_INDEX_JOIN_FIELDS","NO_REBUILD_INDEX")

# export final out 

fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(streetcondition)

analysisfields = ['SUM_OCIxLen','SUM_segmentlength','FREQUENCY']

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
arcpy.conversion.ExportFeatures(streetcondition,"ArlingtonMarketReadinessStreetCondition",
                                "","NOT_USE_ALIAS",fieldmapping,None)

# Alterfield name 
arcpy.management.AlterField("ArlingtonMarketReadinessStreetCondition","WeightedOCI",
                            "StreetCondition","StreetCondition")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\StreetCondition2024'>

- **Zoning**

In [ ]:
arcpy.analysis.Intersect([zoning, boundaries],zoning_in_boundaries,"ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)